# Inferring

LLMs can not only do text generation and summarization, but also sentiment analysis. So you can deploy one model and run multiple kinds of tasks on it.

## Setup

In [1]:
!pip install openai
!pip install python-dotenv

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

# load ENVs from a local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
model = "gpt-3.5-turbo"

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0, # the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Run inference on a piece of review text

### The review text

In [4]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

### Sentiment (positive/negative)

What is the sentiment of the given text? Give your answer as a single word, either "positive" or "negative".

In [5]:
prompt = f"""
What is the sentiment of the following product review, which is delimited with triple backticks?
Give your answer as a single word, either "positive" or "negative".

Review text: ```{lamp_review}```
"""

response = get_completion(prompt)
print(response)

positive


### Identify types of emotions
Identify a list of emotions that the writer is expressing. No more than five items in the list.

In [6]:
prompt = f"""
Identify a list of emotions expressed in the product review, which is delimited with triple backticks.
Include no more than five items in the list. Format your answer as lower-case words separated by comas.

Review text: ```{lamp_review}```
"""

response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


### Identify anger
Is the review expressing anger?

In [8]:
prompt = f"""
Does the following product review express anger? Give your answer as either yes or no.

Review text: ```{lamp_review}```
"""

response = get_completion(prompt)
print(response)

No


### Extract product and company names from customer reviews

In [9]:
prompt = f"""
Extract the following information from the product review text delimited by three backticks:
- purchased item
- company that produced the item

Make your answer as short as possible. Make the output in JSON format with "Iten" and "Company" as the keys. \
    If the information isn't present, use "unknown" as the value.

Review text: ```{lamp_review}```
"""

response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Company": "Lumina"
}


### Do multiple tasks at once
- sentiment (positive or negative)
- anger
- item
- company

In [10]:
prompt = f"""
Identify the following information from the product review text, which is delimited by three backticks:
- sentiment (positive or negative)
- is anger expressed in the review (true or false)
- purchased item
- company that produced the item

Make your answer as short as possible. 
Make the output in JSON format with "Sentiment", "Anger", "Iten" and "Company" as the keys. 
Format the Anger value as a boolean.
If the information isn't present, use "unknown" as the value.

Review text: ```{lamp_review}```
"""

response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Company": "Lumina"
}


## Infer topics

### The story

In [11]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

### Infer 5 topics

In [12]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


### Capture topic and make news alert

In [13]:
topic_list = [
    "nasa", "local goverment", "engineering", 
    "employee satisfaction", "federal government"
]

In [14]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [15]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!
